In [3]:
# Refactoring NASEM model
import nasem_dairy as nd
import pandas as pd
import math

def run_NASEM():
########################################
# Step 1: Read User Input
########################################
    # animal_input is a dictionary with all animal specific parameters
    # diet_info is a dataframe with the user entered feed ingredients and %DM intakes
    user_diet, animal_input, equation_selection = nd.read_csv_input('./input.csv')
        
    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    list_of_feeds = user_diet['Feedstuff'].tolist()
    feed_data = nd.fl_get_rows(list_of_feeds, '../../src/nasem_dairy/data/diet_database.db')
    feed_data.reset_index(inplace=True)
    feed_data = feed_data.rename(columns={"Fd_Name": "Feedstuff"})

    diet_info_initial = pd.DataFrame({'Feedstuff': user_diet['Feedstuff']})
    diet_info_initial = diet_info_initial.merge(feed_data, how='left', on='Feedstuff')

    # Add Fd_DMInp to the diet_info dataframe
    Fd_DMInp = user_diet.set_index('Feedstuff')['kg_user'] / user_diet['kg_user'].sum()
    diet_info_initial.insert(1, 'Fd_DMInp', Fd_DMInp.reindex(diet_info_initial['Feedstuff']).values)
    diet_info_initial['Fd_DMIn'] = diet_info_initial['Fd_DMInp'] * animal_input['DMI']      # Should this be done after DMI equations?

    # Add Fd_DNDF48 column, need to add to the database
    diet_info_initial['Fd_DNDF48'] = 0  

    # Calculate additional physiology values
    animal_input['An_PrePartDay'] = animal_input['An_GestDay'] - animal_input['An_GestLength']
    animal_input['An_PrePartWk'] = animal_input['An_PrePartDay'] / 7

    del(list_of_feeds, Fd_DMInp)

########################################
# Step 2: DMI Equations
########################################
    # # Need to precalculate Dt_NDF for DMI predicitons, this will be based on the user entered DMI (animal_input['DMI])
    Dt_NDF = (diet_info_initial['Fd_NDF'] * diet_info_initial['Fd_DMInp']).sum()

    if equation_selection['DMIn_eqn'] == 0:
        # print('Using user input DMI')
        pass

    # Predict DMI for lactating cow
    elif equation_selection['DMIn_eqn'] == 8: 
        # print("using DMIn_eqn: 8")
        animal_input['DMI'] = nd.calculate_Dt_DMIn_Lact1(
            animal_input['An_Parity_rl'], 
            animal_input['Trg_MilkProd'], 
            animal_input['An_BW'], 
            animal_input['An_BCS'],
            animal_input['An_LactDay'], 
            animal_input['Trg_MilkFatp'], 
            animal_input['Trg_MilkTPp'], 
            animal_input['Trg_MilkLacp'])

    # Predict DMI for heifers    
    elif equation_selection['DMIn_eqn'] in [2,3,4,5,6,7,12,13,14,15,16,17]:
        animal_input['DMI'] = nd.heifer_growth(
            equation_selection['DMIn_eqn'], 
            # diet_info.loc['Diet', 'Fd_NDF'],
            Dt_NDF, 
            animal_input['An_BW'], 
            animal_input['An_BW_mature'], 
            animal_input['An_PrePartWk'], 
            nd.coeff_dict)

    
    elif equation_selection['DMIn_eqn'] in [10,11]:
        animal_input['DMI'] = nd.dry_cow_equations(
            equation_selection['DMIn_eqn'], 
            animal_input['An_BW'], 
            animal_input['An_PrePartWk'], 
            animal_input['An_GestDay'], 
            animal_input['An_GestLength'], 
            Dt_NDF, 
            nd.coeff_dict)
        
    else:
        # It needs to catch all possible solutions, otherwise it's possible that it stays unchanged without warning
        print("DMIn_eqn uncaught - DMI not changed. equation_selection[DMIn_eqn]: "+ str(equation_selection['DMIn_eqn']) )

    del(Dt_NDF)

########################################
# Step 3: Feed Based Calculations
########################################
    diet_info = nd.calculate_diet_info(animal_input['DMI'],
                                       animal_input['An_StatePhys'],
                                       equation_selection['Use_DNDF_IV'],
                                       diet_info=diet_info_initial,
                                       coeff_dict=nd.coeff_dict)
    
    return animal_input, diet_info, equation_selection 

### RUN MODEL ###
animal_input, diet_info, equation_selection = run_NASEM()


C:\Users\Braed\OneDrive - University of Guelph\NASEM-Model-Python\src\nasem_dairy\NASEM_equations\dev_nutrient_intakes.py:807: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_diet_info[f"{column_name}In"] = complete_diet_info.apply(lambda row: row['Fd_DMIn'] * row[f"{column_name}"], axis=1)
C:\Users\Braed\OneDrive - University of Guelph\NASEM-Model-Python\src\nasem_dairy\NASEM_equations\dev_nutrient_intakes.py:807: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_diet_info[f"{column_name}In"] = complete_diet_info.ap

In [ ]:
# Refactor calculate_Du_MiN_NRC2021_g

def calculate_Rum_dcNDF(Dt_NDFIn, Dt_DMIn, Dt_StIn, Dt_CPIn, Dt_ADFIn, Dt_ForWet):
    Rum_dcNDF = -31.9 + 0.721 * Dt_NDFIn / Dt_DMIn * 100 - \
            0.247 * Dt_StIn / Dt_DMIn * 100 + \
            6.63 * Dt_CPIn / Dt_DMIn * 100 - \
            0.211 * (Dt_CPIn / Dt_DMIn * 100) ** 2 - \
            0.387 * Dt_ADFIn / Dt_DMIn / (Dt_NDFIn / Dt_DMIn) * 100 - \
            0.121 * Dt_ForWet + 1.51 * Dt_DMIn

    if Rum_dcNDF < 0.1 or Rum_dcNDF is None:                                                # Line 984
        Rum_dcNDF = 0.1
    return Rum_dcNDF

def calculate_Rum_DigNDFIn(Rum_dcNDF, Dt_NDFIn):
    Rum_DigNDFIn = Rum_dcNDF / 100 * Dt_NDFIn
    return Rum_DigNDFIn

def calculate_Rum_dcSt(Dt_DMIn, Dt_ForNDF, Dt_StIn, Dt_ForWet):
    Rum_dcSt = 70.6 - 1.45*(Dt_DMIn) + 0.424*Dt_ForNDF + \
            1.39*(Dt_StIn)/(Dt_DMIn)*100 - \
            0.0219*((Dt_StIn)/(Dt_DMIn)*100)**2 - \
            0.154*Dt_ForWet

    if Rum_dcSt < 0.1:                                                                      # Line 992
        Rum_dcSt = 0.1            
    elif Rum_dcSt > 100:                                                                    # Line 993
        Rum_dcSt = 100 
    return Rum_dcSt

def calculate_Rum_DigStIn(Rum_dcSt, Dt_StIn):
    Rum_DigStIn = Rum_dcSt / 100 * Dt_StIn                                                   # Line 998
    return Rum_DigStIn

def calculate_Du_MiN_NRC2021_g(MiN_Vm, Rum_DigNDFIn, Rum_DigStIn, An_RDPIN_g, coeff_dict):
    req_coeffs = ['KmMiNRDNDF', 'KmMiNRDSt']
    nd.check_coeffs_in_coeff_dict(coeff_dict, req_coeffs)

    Du_MiN_NRC2021_g = MiN_Vm / (1 + coeff_dict['KmMiNRDNDF'] / Rum_DigNDFIn + coeff_dict['KmMiNRDSt'] / Rum_DigStIn)   # Line 1126

    if Du_MiN_NRC2021_g > 1 * An_RDPIN_g/6.25:  # Line 1130
        Du_MiN_NRC2021_g = 1 * An_RDPIN_g/6.25
    else:
        Du_MiN_NRC2021_g = Du_MiN_NRC2021_g

    return Du_MiN_NRC2021_g

def calculate_Du_MiN_VTln_g(Dt_rOMIn, Dt_ForNDFIn, Rum_DigStIn, Rum_DigNDFIn, coeff_dict):
    req_coeffs = ['Int_MiN_VT',
                  'KrdSt_MiN_VT', 
                  'KrdNDF_MiN_VT', 
                  'KRDP_MiN_VT', 
                  'KrOM_MiN_VT', 
                  'KForNDF_MiN_VT', 
                  'KrOM2_MiN_VT', 
                  'KrdStxrOM_MiN_VT', 
                  'KrdNDFxForNDF_MiN_VT'
                  ]
    nd.check_coeffs_in_coeff_dict(coeff_dict, req_coeffs)

    # Line 1144-1146
    Du_MiN_VTln_g = coeff_dict['Int_MiN_VT'] + coeff_dict['KrdSt_MiN_VT'] * Rum_DigStIn + coeff_dict['KrdNDF_MiN_VT'] * Rum_DigNDFIn 
    + coeff_dict['KRDP_MiN_VT'] * An_RDPIn + coeff_dict['KrOM_MiN_VT'] * Dt_rOMIn + coeff_dict['KForNDF_MiN_VT'] * Dt_ForNDFIn 
    + coeff_dict['KrOM2_MiN_VT'] * Dt_rOMIn ** 2 
    + coeff_dict['KrdStxrOM_MiN_VT'] * Rum_DigStIn * Dt_rOMIn + coeff_dict['KrdNDFxForNDF_MiN_VT'] * Rum_DigNDFIn * Dt_ForNDFIn

    return Du_MiN_VTln_g

def calculate_Du_MiN_VTnln_g(An_RDPIn, Rum_DigNDFIn, Rum_DigStIn):
    Du_MiN_VTnln_g = 7.47 + 0.574 * An_RDPIn * 1000 / (1 + 3.60 / Rum_DigNDFIn + 12.3 / Rum_DigStIn)    # Line 1147
    return Du_MiN_VTnln_g



# Refactor calculate_Du_MiN_g
# Need and equation selection variable, Don't need a function for Du_MiN_g
# There are 3 different equations to choose from, include equation selection here
    # MiN_eqn is the variable name

# if eqn_select == 1:
#     Du_MiN_g = calculate_Du_MiN_NRC2021_g(MiN_Vm, Rum_DigNDFIn, Rum_DigStIn, An_RDPIN_g, nd.coeff_dict)        
# elif eqn_select == 2:
#     Du_MiN_g = calculate_Du_MiN_VTln_g(Dt_rOMIn, Dt_ForNDFIn, Rum_DigStIn, Rum_DigNDFIn, nd.coeff_dict)
# elif eqn_select ==3:
#     Du_MiN_g = calculate_Du_MiN_VTnln_g(An_RDPIn, Rum_DigNDFIn, Rum_DigStIn)
# else:
#     print("Invalid equation number")
#     # Should raise error here



In [ ]:
from icecream import ic

# Test new functions
An_RDPIn = 1   # Set An_RDPIn
An_RDPIN_g = An_RDPIn * 1000
    # There are a lot of these unit coversions to g, should try and do them all at once if possible
Dt_NDFIn = 8.10166182432373
Dt_DMIn = 24.521
Dt_StIn = 4.96654799368879
Dt_CPIn = 5.16656423020988
Dt_ADFIn = 5.62724383705589
Dt_ForWet = 22.3214307767857
Dt_ForNDF = 23.4844869990251

Rum_dcNDF = ic(calculate_Rum_dcNDF(Dt_NDFIn, Dt_DMIn, Dt_StIn, Dt_CPIn, Dt_ADFIn, Dt_ForWet))# Start by testing all functions with values from R

Rum_DigNDFIn = ic(calculate_Rum_DigNDFIn(Rum_dcNDF, Dt_NDFIn))

Rum_dcSt = ic(calculate_Rum_dcSt(Dt_DMIn, Dt_ForNDF, Dt_StIn, Dt_ForWet))

Rum_DigStIn = ic(calculate_Rum_DigStIn(Rum_dcSt, Dt_StIn))